In [5]:
'''Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''

from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

filename = "Demo2.txt"
text = open(filename).read()
text = text.lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


corpus length: 1403671
total chars: 64
nb sequences: 467877
Vectorization...
Build model...


In [6]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])


Epoch 1/60
467877/467877 [==============================] - 588s 1ms/step - loss: 2.4908

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "centre of one
of his problems, and she i"
centre of one
of his problems, and she in and hank and and hand and and and and and has and and and shand and and and and and and and and and and and hand and and and and and and and and and and and and and and and and and and and and and and hank and hank and and and and and and and and and and and and and and and and and hang hang hand and and and and hank hand and and and has and and and and and hank and hank and and and and and and 
----- diversity: 0.5
----- Generating with seed: "centre of one
of his problems, and she i"
centre of one
of his problems, and she i noun ny hing shas had hank and and and thas and had and on and non kpand in you no kk and suak he and shas fas nom and and and whang hank shas and had dang has and and hounk and bund and hand and and hand ka

/home/kynan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log


ath here the hath hath hath hath the hath the hath hath the has here the hang hath the hath he hang the hath the hath the hand hath hath the here the hand the hath the hath the hath here the hath hath hang the hang hand hath the hath the her hath the hath hath the hath the shere hath hath the hath the the hath the hath the hath the hath hath here hath the here here the graghthe here
----- diversity: 0.5
----- Generating with seed: "u have made your statement very clearly."
u have made your statement very clearly. haght has iter any hish, ing the the the herk his the his thatht the the and the kne he houl, the hath the pook. haghthe the the he shere, has the that i this hath the hath his thenther hastesteen hest, thather his the hanged here the knen the the sheshath hen the hath gere hath and his hister, the kne he here mer hath therenters, hesthister thaster, sham mo he that
meming the not ghas the she ha
----- diversity: 1.0
----- Generating with seed: "u have made your statement very

yomecicuemyy, manod was wa gromew medicen sunacome an wmen hit
wel
----- diversity: 1.2
----- Generating with seed: " perhaps it
would have been better for b"
 perhaps it
would have been better for by excoigucon
ofmim."

“exceew."

"



"
"'to ir, weromey sarenery tamonk
ommingisougg taacce," cavadceuww mingeso ang."

“of yemingacm-croncacimebsich
teaghtelaw,
norcafut
anevew andeesflaciey, whitccw, the
doftesansicaty
walcontyevaymeewtwenme
wismy."
"
cigackenele, wedmey
thawcemonce whave go thigion, ingy my
weck was agawted im and. cre
naidgim, as wold thesloncacignane,t
aad doccodmy

and ase,
Epoch 6/60
467877/467877 [==============================] - 583s 1ms/step - loss: 2.2527

----- Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: "nd we were shown up together to
holmes' "
nd we were shown up together to
holmes' lith," in, in, in, ben, it the, in, in, in, in, in, in, ben, in, in in, in, it the hick,", in,",", in, in, it the cill,",", it in, been," in 

"is briony masint aper pastettett thestten the mudtmont teptin to tem the wep thet teptt tas thettertes suttmetttes thet the tim te tute the the muss put nom tentele the ten tiut thattet an the thesttetter thipthest ot tout tut tuttictes thette tiut tute thett mes muttettes thattiptisce tuthtestepeft toupl muse apste teph mes theet santerting thive't thes thet tas ins wuvesnes the tout wes thest tenter the 
----- diversity: 1.0
----- Generating with seed: "iselle's address?" he asked.

"is briony"
iselle's address?" he asked.

"is briony at
stons,t ian wonittt scotee” apderrete thite, at whomtaoum to paseattiw, mlapspilk, austoep stor stoutter,tette tessem tap i,” wavepoulrder mlter t""crhis mor, ans esete pong
shenestipyd tto tasts-poevons ust wistte her
mas tiun swot tharesw well tould utest man thenasatlmer popestiptours witheum tatttesritokmen i wheen wiminelapirt, pee, t,o mit, witess "pottippupmereveen f pat mu, herpiust. i
----- diversity: 1.2
----- Generating with seed: "iselle

...

“wonderful!” said the country doctor. “yot leat i a ster to all a see a i a last to a stoot a last or a lall a so a lood to a to all all to to a lastlet ood a last toolller to to a see to to east to a last a at a selele a latelo to to to to to to a later a stall toollarle a a later to a sters to a see a sellerst tall it at sat ollliot at a lase to a i to a sat eall all or to to a stle to or to a a to at at sele at it i to eid a as a sa 
----- diversity: 1.0
----- Generating with seed: "“wonderful!” said the country doctor. “y"
“wonderful!” said the country doctor. “yelleataist a dorles to tiret lade tir a soor
a stold titloorald to rotder to
seloiess ood tolds tad lastltist silall to ail to air lits ot to sastetiol all told or ostlo tot selastleait a selealleat i tealler trai a a lite a it to a stil a litsred a ssee daret at dolate at to to a all at it a roartait arde to or a i deed see eas to alloor ellter ai to et sletteld tidteate air ore tidle to treeat i
----- diversity: 1.2
-----

rood addledred is at teast so taded as rall deate are troardarled reardalad so asdaare are araise adered drarterles are
Epoch 49/60
467877/467877 [==============================] - 582s 1ms/step - loss: 6.0130

----- Generating text after Epoch: 48
----- diversity: 0.2
----- Generating with seed: " left shoe, just where the firelight str"
 left shoe, just where the firelight stress to to to stost to to to to to stress stress to to to to to to to to to to to stress to to to to to to stress to to to to to to to to stress to to to to to to to stress to to to to to to to to to to stost to strers to to to to to to to to to to to to to to to to to to to to to to to to to to striostress to to to to to to to to stress to to to to to to to to to to to to stress to to to tided to 
----- diversity: 0.5
----- Generating with seed: " left shoe, just where the firelight str"
 left shoe, just where the firelight stritiositiosess to trooss to tidess to to to stooss to to
stotessiot to stost it seliss 

ess, and the runaways will think that evated o seller to as i trorse
sosarte
seoo toor ooor or elldaide tallt orel
to rots soolated to do to
litted
it is areded
orest ot lait toorsatedeld ollod
toords arst to tot stleded tolsed toled as tress
lessed lorsed
tollosols todder oled reeod eroder as to ed oot a
talleded allded tolloldood tiss ossitiorle losloortess
i
lited a i a selator litairt roate i sor lesior od ored tolled stal all lools
----- diversity: 1.2
----- Generating with seed: "ess, and the runaways will think that ev"
ess, and the runaways will think that evited it a is sot
lite to
so
sot to
stlotsortor ot od ot i seareder
tor silel o to
tart oolat
tor doleds tool
at otesied as to to dole oral ass ssie estosliot o littarlted tall loor strestled liod osedlidiot sslotilst a sollsoler
lastest ost es
soroete as alassesis to loost soor ostors tell tooldollelder is
ood a soodoosls tolarlior is oor ort tall loor orseder at
orsller sted dood told
arled 

to

Epoch 53/60
467877/467877 

riminal, for the revolver lay upon the for i do i siriid i sirard i siriid i did is is i sirid i i sid i it i i sirid i siriid i siriid i so it i sirad i sid i so i sid it i siriid i so i siriid i sid i i so i siriid i sirad i so i so i sirid i siriid i is i sirad i siriid to i sirid i i so i sirid i sid it i siriid i do i siriid i striorior i i sid i i it i so i sirtioriorioriorioriorioriorioriorioriorioriorioriord i sirad i sirtiorior
----- diversity: 0.5
----- Generating with seed: "riminal, for the revolver lay upon the f"
riminal, for the revolver lay upon the forsidiorioril arod i sat to do it i ditioriorioriits i it or a sid i sas i striot lasiot to is do to it i sirid i as aid to arod it it i it i dirist is i strial aid ord to rastiorior i strist a i it a sid i so a rood i it is as a a did i sid i it i sirad or i it i so or i to sid to i so i so is is is i so aid i it it i so is i a sid to i do a to i it i siriid aid it i do or it is i sid is i i sili
----- diversity: 1.0
-----

aid lead to i i did sit to a sad to dit do i
list stral a stode s
a last
ellose do tot a o
do i sald as
o i ssollood i so to a lise a a to
----- diversity: 1.2
----- Generating with seed: "
dressed hurriedly, for i knew by experi"

dressed hurriedly, for i knew by experild leelal i
de sle to ste to i
dess dast i led
sol sall lal selatliotordored ood allt lat at to telese
all i sirlo to osolled ooosis s eit all tell
lit tade
to a dord
lelloolo a s tisad i i lald
do ot dassited so o so sid to aid to o i dolliot olal to
i tisall a lastle so oads sist sood le a stllos lisoor see to reast as did it at a load all i ssad to told is
to too selooet all desss doal
it a
de

Epoch 60/60
467877/467877 [==============================] - 588s 1ms/step - loss: 6.0188

----- Generating text after Epoch: 59
----- diversity: 0.2
----- Generating with seed: " murder. i know how keen you
are upon th"
 murder. i know how keen you
are upon the sall a sis a sistle a siss a sid a lassad a sis at a sist a silad